In [20]:
import pandas as pd
from time import time
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [159]:
data = pd.read_csv('data/2018/201806-citibike-tripdata.csv')

In [126]:
weather = pd.read_csv('data/weather/201806.csv')

In [127]:
weather.head(5)

,Local time in New York / La Guardia (airport),T,Po,P,Pa,U,DD,Ff,ff10,ff3,...,Cm,Ch,VV,Td,RRR,tR,E,Tg,E',sss
0,30.06.2018 22:00,28.3,760.7,761.8,-0.2,57,Wind blowing from the south,3,NaN,NaN,...,NaN,NaN,16.0,18.9,NaN,NaN,NaN,NaN,NaN,NaN
1,30.06.2018 19:00,31.7,760.9,761.8,-0.3,39,Wind blowing from the south-southeast,5,NaN,NaN,...,NaN,NaN,16.0,16.1,NaN,NaN,NaN,NaN,NaN,NaN
2,30.06.2018 16:00,34.4,761.2,762.3,-0.6,30,Wind blowing from the south-west,5,NaN,NaN,...,NaN,NaN,16.0,14.4,NaN,NaN,NaN,NaN,NaN,NaN
3,30.06.2018 13:00,33.9,761.8,762.8,0.3,31,Wind blowing from the south,5,NaN,NaN,...,NaN,NaN,16.0,14.4,NaN,NaN,NaN,NaN,NaN,NaN
4,30.06.2018 10:00,30.6,761.5,762.6,-1.0,40,Wind blowing from the north-east,3,NaN,NaN,...,NaN,NaN,16.0,15.6,NaN,NaN,NaN,NaN,NaN,NaN


In [128]:
# T U N RRR
weather["Time"] = weather["Local time in New York / La Guardia (airport)"]
del weather["Local time in New York / La Guardia (airport)"]
del weather["Po"]
del weather["P"]
del weather["Pa"]
del weather["DD"]
del weather["ff10"]
del weather["ff3"]
del weather["WW"]
del weather["W1"]
del weather["W2"]
del weather["Tx"]
del weather["Tn"]
del weather["Cl"]
del weather["Nh"]
del weather["H"]
del weather["Cm"]
del weather["Ch"]
del weather["VV"]
del weather["Td"]
del weather["tR"]
del weather["E"]
del weather["Tg"]
del weather["E'"]
del weather["sss"]

In [129]:
weather.head(5)

,T,U,Ff,N,RRR,Time
0,28.3,57,3,20–30%.,NaN,30.06.2018 22:00
1,31.7,39,5,20–30%.,NaN,30.06.2018 19:00
2,34.4,30,5,20–30%.,NaN,30.06.2018 16:00
3,33.9,31,5,20–30%.,NaN,30.06.2018 13:00
4,30.6,40,3,20–30%.,NaN,30.06.2018 10:00


In [130]:
def parse_clouds(clouds):
    if clouds == "no clouds":
        return 0
    else:
#         print(type(clouds[:-2].split("–")))
        clouds = [int(i.strip()) for i in clouds[:-2].split("–")]
        return int(sum(clouds) / len(clouds))

In [131]:
def parse_precipation(p):
    if(pd.isna(p)):
        return 0
    if p == "Trace of precipitation":
        return 0.15
    else:
        return p

In [132]:
def get_time(time_str):
    day = int(time_str[:2])
    int_time = int(time_str[11:13])
    if 0 <= int_time and int_time < 6:
        return [day, "night"]
    if 6 <= int_time and int_time < 12:
        return [day, "morning"]
    if 12 <= int_time and int_time < 18:
        return [day, "afternoon"]
    if 18 <= int_time and int_time <= 23:
        return [day, "evening"]

In [158]:
def get_time2(time_str):
    day = int(time_str[8:10])
    int_time = int(time_str[11:13])
    if 0 <= int_time and int_time < 6:
        return [day, "night"]
    if 6 <= int_time and int_time < 12:
        return [day, "morning"]
    if 12 <= int_time and int_time < 18:
        return [day, "afternoon"]
    if 18 <= int_time and int_time <= 23:
        return [day, "evening"]

In [142]:
from math import sin, cos, sqrt, atan2, radians

def get_distance(lat1, lon1, lat2, lon2):
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return round(distance, 2)

In [133]:
precipation = [] 
clouds = []
times = []
days = []
c=time()

for i in range(len(weather)):
    clouds.append(parse_clouds(weather["N"][i]))
    precipation.append(parse_precipation(weather["RRR"][i]))
    parsed_time = get_time(weather["Time"][i])
    times.append(parsed_time[1])
    days.append(parsed_time[0])
weather['clouds'] = pd.Series(clouds)
weather['precipation'] = pd.Series(precipation)
weather['Time'] = pd.Series(times)
weather['Day'] = pd.Series(days)
del weather["N"]
del weather["RRR"]
print(time() - c)

0.042482614517211914


In [134]:
weather.head(5)

,T,U,Ff,Time,clouds,precipation,Day
0,28.3,57,3,evening,25,0,30
1,31.7,39,5,evening,25,0,30
2,34.4,30,5,afternoon,25,0,30
3,33.9,31,5,afternoon,25,0,30
4,30.6,40,3,morning,25,0,30


In [160]:
data.head(5)

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,569,2018-06-01 01:57:20.5140,2018-06-01 02:06:50.0880,72,W 52 St & 11 Ave,40.767272,-73.993929,173,Broadway & W 49 St,40.760683,-73.984527,21481,Subscriber,1999,1
1,480,2018-06-01 02:02:42.3980,2018-06-01 02:10:43.3540,72,W 52 St & 11 Ave,40.767272,-73.993929,477,W 41 St & 8 Ave,40.756405,-73.990026,19123,Subscriber,1988,1
2,692,2018-06-01 02:04:23.6240,2018-06-01 02:15:55.7470,72,W 52 St & 11 Ave,40.767272,-73.993929,457,Broadway & W 58 St,40.766953,-73.981693,26983,Subscriber,1986,1
3,664,2018-06-01 03:00:55.4610,2018-06-01 03:11:59.9060,72,W 52 St & 11 Ave,40.767272,-73.993929,379,W 31 St & 7 Ave,40.749156,-73.991600,26742,Subscriber,1973,1
4,818,2018-06-01 06:04:54.4270,2018-06-01 06:18:32.6170,72,W 52 St & 11 Ave,40.767272,-73.993929,459,W 20 St & 11 Ave,40.746745,-74.007756,26386,Subscriber,1984,1


In [161]:
from time import time
data1 = data[0:100000]
times = []
days = []
distances = []
c=time()

for i in range(len(data1)):
#     print(i)
    time_i = get_time2(data1["starttime"][i])
    times.append(time_i[1])
    days.append(time_i[0])
    distances.append(get_distance(data1["start station latitude"][i], data1["start station longitude"][i], data1["end station latitude"][i], data1["end station longitude"][i]))
    
data1["time"] = pd.Series(times)
data1["distance"] = pd.Series(distances)
data1["day"] = pd.Series(days)

print(time() - c)

13.362556457519531


c:\users\user1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
c:\users\user1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
c:\users\user1\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

In [162]:
del data1["starttime"]
del data1["stoptime"]
del data1["start station id"]
del data1["start station name"]
del data1["end station id"]
del data1["end station name"]
del data1["bikeid"]
del data1["start station latitude"]
del data1["start station longitude"]
del data1["end station latitude"]
del data1["end station longitude"]

In [163]:
data1.head(5)

,tripduration,usertype,birth year,gender,time,distance,day
0,569,Subscriber,1999,1,night,1.08,1
1,480,Subscriber,1988,1,night,1.25,1
2,692,Subscriber,1986,1,night,1.03,1
3,664,Subscriber,1973,1,night,2.02,1
4,818,Subscriber,1984,1,morning,2.56,1


In [ ]:
#TODO: connect weather and data1